In [1]:
import slideio
import json
import matplotlib.path as mpltPath
import warnings
import tifffile
import staintools
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
import cv2
import glob
import tensorflow as tf
import tensorflow.keras as K
import os
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import PIL
import PIL.Image
from sklearn.model_selection import train_test_split,StratifiedKFold
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

/home/dgxadmin/anaconda3/envs/LeeYS/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
tfds.disable_progress_bar()
autotune = tf.data.experimental.AUTOTUNE
def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    # Map values in the range [-1, 1]
    return (img / 127.5) - 1.0

def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    # Map values in the range [-1, 1]
    return (img / 127.5) - 1.0


def preprocess_train_image(img):
    # Random flip
    # Resize to the original size first
    # Normalize the pixel values in the range [-1, 1]
    img = normalize_img(img)
    return img


def preprocess_test_image(img, label):
    # Only resizing and normalization for the test images.
    img = tf.image.resize(img, [input_img_size[0], input_img_size[1]])
    img = normalize_img(img)
    return img

input_img_size = (256,256, 3)

orig_img_size = (286, 286)
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

buffer_size = 256
batch_size = 1
# calculate frechet inception distance
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="valid",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    dim = x.shape[-1]
    input_tensor = x

    x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    x = ReflectionPadding2D()(x)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.add([input_tensor, x])
    return x


def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x

def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = ReflectionPadding2D(padding=(3, 3))(img_input)
    x = layers.Conv2D(filters, (7, 7), kernel_initializer=kernel_init, use_bias=False)(
        x
    )
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(3, (7, 7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model

def get_discriminator(
    filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(
        filters,
        (4, 4),
        strides=(2, 2),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(
        1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer
    )(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model


# Get the generators
gen_G = get_resnet_generator(name="generator_G")
gen_F = get_resnet_generator(name="generator_F")

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")

class CycleGan(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle=10.0,
        lambda_identity=0.5,
    ):
        super().__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
    ):
        super().compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, batch_data):
        # x is Horse and y is zebra
        real_x, real_y = batch_data

        # For CycleGAN, we need to calculate different
        # kinds of losses for the generators and discriminators.
        # We will perform the following steps here:
        #
        # 1. Pass real images through the generators and get the generated images
        # 2. Pass the generated images back to the generators to check if we
        #    we can predict the original image from the generated image.
        # 3. Do an identity mapping of the real images using the generators.
        # 4. Pass the generated images in 1) to the corresponding discriminators.
        # 5. Calculate the generators total loss (adverserial + cycle + identity)
        # 6. Calculate the discriminators loss
        # 7. Update the weights of the generators
        # 8. Update the weights of the discriminators
        # 9. Return the losses in a dictionary

        with tf.GradientTape(persistent=True) as tape:
            # Horse to fake zebra
            fake_y = self.gen_G(real_x, training=True)
            # Zebra to fake horse -> y2x
            fake_x = self.gen_F(real_y, training=True)

            # Cycle (Horse to fake zebra to fake horse): x -> y -> x
            cycled_x = self.gen_F(fake_y, training=True)
            # Cycle (Zebra to fake horse to fake zebra) y -> x -> y
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adverserial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(tf.ones_like(fake), fake)
    return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(tf.ones_like(real), real)
    fake_loss = adv_loss_fn(tf.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5
cycle_gan_model = CycleGan(
        generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
    )
cycle_gan_model.compile(
        gen_G_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
        gen_F_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
        disc_X_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
        disc_Y_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
        gen_loss_fn=generator_loss_fn,
        disc_loss_fn=discriminator_loss_fn,
    )
cycle_gan_model.built = True
cycle_gan_model.load_weights("../model/CycleGan/patition/cyclegan_checkpoints_016.h5")

In [3]:
MMRstatus_data=pd.read_csv('../MMRstatus.csv',encoding='cp949')
P53_data=pd.read_csv('../P53.csv',encoding='cp949')

In [4]:
def svsToTiff(imageCount,magnification,tile_size,slideName):
    global svsWidth,svsHeight,path,scene,fileName
    reduction_rate=360
    slide_tile_size=4096
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)>0.7
            Full=np.mean(points)>0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(1024,1024))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(1024*1024)/8:
                    TiffImageSave(image, fileName, imageCount,slide_tile_size,slideName)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,imageCount,slide_tile_size,slideName):
    dst_img=np.zeros(image1.shape,dtype=np.uint8)
    for i in range(4):
        for j in range(4):
            image2=image1[256*i:256+256*i,256*j:256+256*j]
            image2=np.reshape(image2,(1,256,256,3))
            image2=tf.convert_to_tensor(image2)
            image2=cycle_gan_model.gen_G(image2)[0].numpy()
            image2=(image2*127.5+127.5).astype(np.uint8)
            dst_img[256*i:256+256*i,256*j:256+256*j]=image2
    dst_img=cv2.cvtColor(dst_img, cv2.COLOR_BGR2RGB)
    cv2.imwrite('../data/CycleGanTiling/'+ slideName + "/" + fileName + "_" + str(
        imageCount)+ '.tiff', dst_img)
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
    

In [5]:
warnings.filterwarnings(action='ignore')
files = os.listdir('../svs_data/')
svs_files=[]
xml_files=[]
for i in range(len(files)):
    svs_class=files[i].find('.svs')
    xml_class=files[i].find('.xml')
    if svs_class!=-1:
        svs_files.append(files[i])
    elif xml_class!=-1:
        xml_files.append(files[i])

In [6]:
for i in range(len(svs_files)):
    try:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        slideName=svs_files[i][:9]
        slide = slideio.open_slide('../svs_data/'+fileName+'.svs', "SVS")
        scene = slide.get_scene(0)
        svsWidth = scene.rect[2]
        svsHeight = scene.rect[3]
        imageCount=0
        magnification=80
        tile_size=256
        createDirectory('../data/CycleGanTiling/' + slideName)
        xml_open='../svs_data/'+fileName+'.xml'
        polygon_xml = ET.parse(xml_open)
        root = polygon_xml.getroot()
        root = root.getchildren()
        root =root[0].getchildren()
        imageCount=0
        for annotation_count in range(len(root)):
            root1 =root[annotation_count].getchildren()
            root1 =root1[0].getchildren()
            polygon=[]
            for polygon_count in range(len(root1)): 
                polygon.append([int(float(root1[polygon_count].attrib['X'])),int(float(root1[polygon_count].attrib['Y']))])
            path = mpltPath.Path(polygon)
            imageCount=svsToTiff(imageCount,magnification,tile_size,slideName)
    except:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        print(fileName)

S10-01345-1
S10-06438-2
S10-06438-16
S10-07753-6
S10-27147-4R
S10-27147-5R
S10-28950-7
S10-28950-8
S11-13522-9
S11-14059-1
S11-15498-9
S11-18614-2
S11-23853-MG4
S11-25637-12
S11-25637-15
S11-27049-1
S11-27049-2
S11-27296-2
S11-27296-3
S11-27296-4
S11-27749-15
S11-31465-15
S11-33397-8R
S12-02539-7
S12-05129-1
S12-05129-9
S12-15923-MG8
S12-18293-4
S12-25754-12
S12-26702-1
S12-30332-16
S12-30332-17
S13-16862-7
S13-17955-3
S13-17955-11
S13-17955-12
S13-19937-23
S13-23875-16R
S14-04726-2
S14-04976-5
S14-04976-8
S14-05313-4
S14-05313-5
S14-06111-6
S14-11763-1
S14-11763-2
S14-13997-2
S14-21094-2
S14-21094-3
S14-27557-5
S14-29043-3
S14-29043-4
S15-01797-1
S15-02984-36
S15-03278-7
S15-03278-8
S15-03278-9
S15-03845-8
S15-04670-1
S15-14451-2
S15-14532-MR3
S15-15513-4
S15-15883-1
S15-15883-27
S15-15883-28
S15-16222-7
S15-18047-2R
S15-25474-14
S15-25474-18
S15-25474-19
S16-04113-10
S16-04113-14
S16-06596-10
S16-07528-3
S16-08284-11
S16-09342-4
S16-10297-15
S16-13007-12
S16-13063-11
S16-13989-10
S16